- *Observed Trend 1*: Based on overall passing rates, all of the top five schools are charter schools, and all of the bottom five schools are district schools. Charter schools also outperform district schools in average math score, average reading score, percent passing math, and percent passing reading.
- *Observed Trend 2*: Which school a student attended appeared to have more of an impact on average math and reading scores than grade level. School size also appeared to influence average math and reading scores, as well as percent passing math and reading and overall passing rate, with students at small schools (less than 1,700 total students) faring better than those at medium or large schools.
- *Observed  Trend 3*: Per student budget was not a strong indicator of student performance (as measured by average math and reading scores, percent passing math and reading, and overall passing rate). Students at schools with per student budgets less than \$615 per student scored higher in each of these categories than those at schools with per student budgets above \$615 dollars per student.

**Conclusion**: School size and type appeared to be the best indicators for student performance based on these data.

**Future Questions**: How do charter schools compare in size to district schools? It is possible that students at charter schools perform better because those schools are smaller than district schools, but more analysis is needed before we can determine whether that is true.

# Getting Started: Importing Dependencies and Data

In [1]:
# Dependencies
import pandas as pd

In [2]:
# Importing raw data
csvpath1 = 'raw_data/schools_complete.csv'
csvpath2 = 'raw_data/students_complete.csv'

schools_df = pd.read_csv(csvpath1)
students_df = pd.read_csv(csvpath2)

In [3]:
# Preview schools DataFrame
schools_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [4]:
# Preview students DataFrame
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


# District Summary

Create a high level snapshot (in table form) of the district's key metrics, including:
- Total Schools
- Total Students
- Total Budget
- Average Math Score
- Average Reading Score
- % Passing Math
- % Passing Reading
- Overall Passing Rate (Average of the above two)

In [5]:
# Calculating total schools, total students, total budget, avg. math score, and avg. reading score
total_schools = len(schools_df)
total_students = len(students_df)
total_budget = schools_df['budget'].sum()
avg_math = students_df['math_score'].mean()
avg_reading = students_df['reading_score'].mean()

#Creating dataframes with only students passing math and only students passing reading
passing_math = students_df[students_df['math_score'] >= 60]
passing_reading = students_df[students_df['reading_score'] >= 60]

# Calculating % passing in reading and math. Scores of 60 or above = passing
percent_passing_math = passing_math['math_score'].sum() / total_students
percent_passing_reading = passing_reading['reading_score'].sum() / total_students

# Adding a column for overall score (avg. of reading and math scores)
for student in students_df:
    students_df['overall_score'] = (students_df['reading_score'] + students_df['math_score']) / 2
    
# Creating a datframe with only passing students and calculating overall passing rate
passing_overall = students_df[students_df['overall_score'] >= 60]
percent_passing = passing_overall['overall_score'].sum() / total_students

In [6]:
# Making table with total schools, total students, total budget, avg. math score,
# avg. reading score, % passing math, % passing reading, and % passing overall.
district_summary = pd.DataFrame({'Total Schools':total_schools,
                                 'Total Students':total_students,
                                 'Total Budget':total_budget,
                                 'Average Math Score':avg_math,
                                 'Average Reading Score':avg_reading,
                                 '% Passing Math':percent_passing_math,
                                 '% Passing Reading':percent_passing_reading,
                                 'Overall Passing Rate':percent_passing}, index=[0])

# Reordering columns to match preferred output format.
district_summary = district_summary[['Total Schools', 'Total Students','Total Budget', 'Average Math Score',
                  'Average Reading Score', '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]

# Printing district summary
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.678555,81.87784,80.342213


# School Summary

Create an overview table that summarizes key metrics about each school, including:
- School Name
- School Type
- Total Students
- Total School Budget
- Per School Budget
- Average Math Score
- Average Reading Score
- % Passing Math
- % Passing Reading
- Overall Passing Rate (Average of the above two)

In [7]:
# Pulling subset of columns from schools_df to start school summary dataframe, then renaming columns.
school_summary = schools_df[['name', 'type', 'size', 'budget']]
school_summary = school_summary.rename(columns={'name':'School',
                                                'type':'School Type',
                                                'size':'Total Students',
                                                'budget':'Total School Budget'})

# Setting the index to the school name
school_summary.set_index(school_summary['School'], inplace=True)
#Dropping duplicate "School" column
school_summary = school_summary[['School Type', 'Total Students', 'Total School Budget']]

# Adding a column with per student budget
school_summary['Per Student Budget'] = school_summary['Total School Budget'] / school_summary['Total Students']

# Grouping students_df by school so we can summarize info from students_df for each school
grouped_students = students_df.groupby(['school'])

# Adding columns with average math and reading scores
school_summary['Average Math Score'] = grouped_students['math_score'].mean()
school_summary['Average Reading Score'] = grouped_students['reading_score'].mean()

# Grouping the dataframes with students passing math, reading, and overall by school so we can
# calculate % passing math, reading, and overall for each school
passing_math_grouped = passing_math.groupby('school')
passing_reading_grouped = passing_reading.groupby('school')
passing_overall_grouped = passing_overall.groupby('school')

# Adding columns for percent passing math, reading, and overall
school_summary['% Passing Math'] = passing_math_grouped['math_score'].sum() / school_summary['Total Students']
school_summary['% Passing Reading'] = passing_reading_grouped['reading_score'].sum() / school_summary['Total Students']
school_summary['Overall Passing Rate'] = passing_overall_grouped['overall_score'].sum() / school_summary['Total Students']

# Printing school summary
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,70.279054,81.182722,78.865444
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,70.120380,81.158020,78.814344
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,83.359455,83.725724,83.542589
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,71.068177,80.934412,78.945415
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,83.351499,83.816757,83.584128
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,83.274201,83.989488,83.631844
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,83.061895,83.975780,83.518837
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,71.079783,81.033963,78.874196
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,83.803279,83.814988,83.809133


# Top 5 Schools by Overall Passing Rate

Create a table highlighting the top 5 performing schools based on overall passing rate.

In [8]:
# Sort school summary by passing rate (high to low), then save only top 5 rows.
school_summary.sort_values('Overall Passing Rate', ascending=False, inplace=True)
top_5_schools = school_summary.iloc[[0, 1, 2, 3, 4]]

# Printing top 5 schools by passing rate.
top_5_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School,,,,,,,,,
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,83.839917,84.044699,83.942308
Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000,83.682222,83.955000,83.818611
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,83.803279,83.814988,83.809133
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,83.418349,83.848930,83.633639
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,83.274201,83.989488,83.631844


# Bottom 5 Schools by Overall Passing Rate

Create a table highlighting the bottom 5 performing schools based on overall passing rate.

In [9]:
# Sort school summary by passing rate (low to high), then save only top 5 rows.
school_summary.sort_values('Overall Passing Rate', inplace=True)
bottom_5_schools = school_summary.iloc[[0, 1, 2, 3, 4]]

# Printing bottom 5 schools by passing rate.
bottom_5_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,70.310078,80.744686,78.689797
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,70.999270,80.746258,78.773092
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,70.120380,81.158020,78.814344
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,70.279054,81.182722,78.865444
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,71.079783,81.033963,78.874196


# Math Scores by Grade

Create a table that lists the average math score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [10]:
# To start, make separate groupby objects for each grade (grouped by school)
grade9 = students_df[students_df['grade']=='9th'].groupby('school')
grade10 = students_df[students_df['grade']=='10th'].groupby('school')
grade11 = students_df[students_df['grade']== '11th'].groupby('school')
grade12 = students_df[students_df['grade']=='12th'].groupby('school')

# Then calculate mean math scores and put into a dataframe
math_scores_by_grade = pd.DataFrame(grade9['math_score'].mean())
math_scores_by_grade.rename(columns={'math_score':'9th Grade'}, inplace=True)
math_scores_by_grade.index.rename('School', inplace=True)
math_scores_by_grade['10th Grade'] = grade10['math_score'].mean()
math_scores_by_grade['11th Grade'] = grade11['math_score'].mean()
math_scores_by_grade['12th Grade'] = grade12['math_score'].mean()

# Print math scores by grade for each school
math_scores_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
School,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


# Reading Scores by Grade

Create a table that lists the average reading score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [11]:
# Repeat the same process used for math scores by grade, but this time for reading.
reading_scores_by_grade = pd.DataFrame(grade9['reading_score'].mean())
reading_scores_by_grade.rename(columns={'reading_score':'9th Grade'}, inplace=True)
reading_scores_by_grade.index.rename('School', inplace=True)
reading_scores_by_grade['10th Grade'] = grade10['reading_score'].mean()
reading_scores_by_grade['11th Grade'] = grade11['reading_score'].mean()
reading_scores_by_grade['12th Grade'] = grade12['reading_score'].mean()

# Print math scores by grade for each school
reading_scores_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
School,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


# Scores by School Spending

Create a table that breaks down school performances based on average Spending Ranges (Per Student).
Use 4 reasonable bins to group school spending. Include in the table each of the following:
- Average Math Score
- Average Reading Score
- % Passing Math
- % Passing Reading
- Overall Passing Rate (Average of the above two)

In [12]:
# First get an idea of good bin sizes to use for the total budgets.
school_summary['Per Student Budget'].describe()

count     15.000000
mean     620.066667
std       28.544368
min      578.000000
25%      591.500000
50%      628.000000
75%      641.500000
max      655.000000
Name: Per Student Budget, dtype: float64

In [13]:
# Set names and boundaries for bins
bin_names = ['< $590', '$590-615', '$615-640', '> $640']
bins = [0, 590, 615, 640, school_summary['Per Student Budget'].max()]

# Now cut schools and place them into bins, then add bin category to schools dataframe
school_summary['Per Student Budget Category'] = pd.cut(school_summary['Per Student Budget'], bins, labels=bin_names)

# Merge budget category into students_df
students_df = pd.merge(students_df, school_summary[['Per Student Budget Category']],
                       how='left', left_on='school', right_index=True)

In [14]:
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score,overall_score,Per Student Budget Category
0,0,Paul Bradley,M,9th,Huang High School,66,79,72.5,> $640
1,1,Victor Smith,M,12th,Huang High School,94,61,77.5,> $640
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,75.0,> $640
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,62.5,> $640
4,4,Bonnie Ray,F,9th,Huang High School,97,84,90.5,> $640


In [15]:
# Grouping students_df by budget category so we can summarize info from students_df for each.
budget_grouped_students = students_df.groupby(['Per Student Budget Category'])

# Adding columns with average math and reading scores
school_summary_by_budget = pd.DataFrame(budget_grouped_students['math_score'].mean()).rename(columns={'math_score':
                                                                                                     'Average Math Score'})
school_summary_by_budget['Average Reading Score'] = budget_grouped_students['reading_score'].mean()

# Recreating dataframes for students passing math, reading, and overall
# so they now include budget category as well.
passing_math = students_df[students_df['math_score'] >= 60]
passing_reading = students_df[students_df['reading_score'] >= 60]
passing_overall = students_df[students_df['overall_score'] >= 60]

# Grouping the dataframes with students passing math, reading, and overall by budget category
# so we can calculate % passing math, reading, and overall for each.
passing_math_budget_grouped = passing_math.groupby('Per Student Budget Category')
passing_reading_budget_grouped = passing_reading.groupby('Per Student Budget Category')
passing_overall_budget_grouped = passing_overall.groupby('Per Student Budget Category')

# Adding columns for percent passing math, reading, and overall
school_summary_by_budget['% Passing Math'] = (passing_math_budget_grouped['math_score'].sum() / 
                                              budget_grouped_students['Student ID'].count())
school_summary_by_budget['% Passing Reading'] = (passing_reading_budget_grouped['reading_score'].sum() / 
                                                 budget_grouped_students['Student ID'].count())
school_summary_by_budget['% Overall Passing Rate'] = (passing_overall_budget_grouped['overall_score'].sum() / 
                                                      budget_grouped_students['Student ID'].count())

# Printing scores by budget category
school_summary_by_budget

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Per Student Budget Category,,,,,
< $590,83.363065,83.964039,83.363065,83.964039,83.663552
$590-615,83.529196,83.838414,83.529196,83.838414,83.683805
$615-640,78.236441,81.559460,73.227990,81.559460,79.791342
> $640,77.058995,80.958411,70.851448,80.958411,78.882507


# Scores by School Size

Repeat the Per Student Budget breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).

In [16]:
# First, get a good sense of what bins to use
school_summary['Total Students'].describe()

count      15.000000
mean     2611.333333
std      1420.915282
min       427.000000
25%      1698.000000
50%      2283.000000
75%      3474.000000
max      4976.000000
Name: Total Students, dtype: float64

In [17]:
# Set names and boundaries for bins
bin_names = ['Small (< 1,700)', 'Medium (1,700-3,400)', 'Large (> 3,400)']
bins = [0, 1700, 3400, school_summary['Total Students'].max()]

# Now cut schools and place them into bins, then add bin category to schools dataframe
school_summary['Size Category'] = pd.cut(school_summary['Total Students'], bins, labels=bin_names)

# Merge school size category into students_df
students_df = pd.merge(students_df, school_summary[['Size Category']],
                       how='left', left_on='school', right_index=True)

In [18]:
# Grouping students_df by school size category so we can summarize info from students_df for each.
size_grouped_students = students_df.groupby(['Size Category'])

# Adding columns with average math and reading scores
school_summary_by_size = pd.DataFrame(size_grouped_students['math_score'].mean()).rename(columns={'math_score':
                                                                                                     'Average Math Score'})
school_summary_by_size['Average Reading Score'] = size_grouped_students['reading_score'].mean()

# Recreating dataframes for students passing math, reading, and overall
# so they now include school size category as well.
passing_math = students_df[students_df['math_score'] >= 60]
passing_reading = students_df[students_df['reading_score'] >= 60]
passing_overall = students_df[students_df['overall_score'] >= 60]

# Grouping the dataframes with students passing math, reading, and overall by size category
# so we can calculate % passing math, reading, and overall for each.
passing_math_size_grouped = passing_math.groupby('Size Category')
passing_reading_size_grouped = passing_reading.groupby('Size Category')
passing_overall_size_grouped = passing_overall.groupby('Size Category')

# Adding columns for percent passing math, reading, and overall
school_summary_by_size['% Passing Math'] = (passing_math_size_grouped['math_score'].sum() / 
                                              size_grouped_students['Student ID'].count())
school_summary_by_size['% Passing Reading'] = (passing_reading_size_grouped['reading_score'].sum() / 
                                                 size_grouped_students['Student ID'].count())
school_summary_by_size['% Overall Passing Rate'] = (passing_overall_size_grouped['overall_score'].sum() / 
                                                      size_grouped_students['Student ID'].count())

# Printing scores by school size category
school_summary_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Size Category,,,,,
"Small (< 1,700)",83.523375,83.877115,83.523375,83.877115,83.700245
"Medium (1,700-3,400)",79.892255,82.396762,76.539155,82.396762,81.090023
"Large (> 3,400)",77.070764,80.928365,70.864297,80.928365,78.857329


# Scores by School Type

Repeat the breakdown above for school size, but this time group schools based on school type (Charter vs. District).

In [19]:
# Merge school type into students_df
students_df = pd.merge(students_df, school_summary[['School Type']],
                       how='left', left_on='school', right_index=True)

In [20]:
# Grouping students_df by school type so we can summarize info from students_df for each.
school_type_grouped_students = students_df.groupby(['School Type'])

# Adding columns with average math and reading scores
school_summary_by_type = pd.DataFrame(school_type_grouped_students['math_score'].mean()).rename(columns={'math_score':
                                                                                                     'Average Math Score'})
school_summary_by_type['Average Reading Score'] = school_type_grouped_students['reading_score'].mean()

# Recreating dataframes for students passing math, reading, and overall
# so they now include school type as well.
passing_math = students_df[students_df['math_score'] >= 60]
passing_reading = students_df[students_df['reading_score'] >= 60]
passing_overall = students_df[students_df['overall_score'] >= 60]

# Grouping the dataframes with students passing math, reading, and overall by school type
# so we can calculate % passing math, reading, and overall for each.
passing_math_school_type = passing_math.groupby('School Type')
passing_reading_school_type = passing_reading.groupby('School Type')
passing_overall_school_type = passing_overall.groupby('School Type')

# Adding columns for percent passing math, reading, and overall
school_summary_by_type['% Passing Math'] = (passing_math_school_type['math_score'].sum() / 
                                              school_type_grouped_students['Student ID'].count())
school_summary_by_type['% Passing Reading'] = (passing_reading_school_type['reading_score'].sum() / 
                                                 school_type_grouped_students['Student ID'].count())
school_summary_by_type['Overall Passing Rate'] = (passing_overall_school_type['overall_score'].sum() / 
                                                      school_type_grouped_students['Student ID'].count())

# Printing scores by school size category
school_summary_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,83.406183,83.902821,83.654502
District,76.987026,80.962485,70.733393,80.962485,78.844955
